In [1]:
import geemap
import ee
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [6]:
roi = ee.FeatureCollection("users/yehuigeo/poyanghu")

In [27]:
def ND_VI(image,b1,b2,bName):  
    VI = image.normalizedDifference([b1,b2]).rename(bName)
    return VI.updateMask(VI.gt(-1).And(VI.lt(1)))
#这里对大于-1小于1的像元进行掩膜

#定义计算哨兵二号的EVI指数的函数
def funEVI(image,a,b,c):   
    VI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'BLUE': image.select(a),   
      'RED': image.select(b),   
      'NIR': image.select(c)}).rename('EVI')
    return VI.updateMask(VI.gt(-1).And(VI.lt(1)))
#定义添加波段的函数
def addLandsatVIs(img):
    NDVI = ND_VI(img,'B8','B4','NDVI')
    EVI = funEVI(img,'B2','B4','B8')
    mNDWI = ND_VI(img,'B3','B6','mNDWI')
    return img.addBands(NDVI).addBands(EVI).addBands(mNDWI)
    # 水提取 MNDWI > NDVI或MNDWI > EVI 和 EVI < 0.1

In [34]:
#丰水期

for year in range(1986,2020):
    for month in list(range(6,10)):
        START_DATE  = str(year) + '-'+ str('%02d' % month)
        END_DATE = str(year) + '-' +str('%02d'% (month+1))
    #get Landsat5 collction B1:blue,B2:green,B3:red,B4:NIR
        collection5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
            .filterBounds(roi) \
            .filterDate(START_DATE,END_DATE ) \
            .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'])\
            .map(addLandsatVIs)
     #get Landsat7 collction B1:blue,B2:green,B3:red,B4:NIR
        collection7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
            .filterBounds(roi) \
            .filterDate(START_DATE, END_DATE ) \
            .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'])\
            .map(addLandsatVIs)
    #get Landsat8 collction
        collection8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
            .filterBounds(roi) \
            .filterDate(START_DATE,END_DATE ) \
            .select( 
              ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa']
              ,['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'])\
            .map(addLandsatVIs)
        # Merge to a single collection 融合成一个数据集
        col1 = ee.ImageCollection(collection5.merge(collection7))
        collection = ee.ImageCollection(col1.merge(collection8))

        #names = collection.aggregate_array('system:index').getInfo()
    #print(names)
    
        print(year,month,collection.size().getInfo())
        print('\n')

1986 6 2
1986 7 3
1986 8 2
1986 9 0
1987 6 2
1987 7 0
1987 8 6
1987 9 4
1988 6 5
1988 7 3
1988 8 4
1988 9 2
1989 6 2
1989 7 4
1989 8 6
1989 9 4
1990 6 3
1990 7 2
1990 8 6
1990 9 2
1991 6 2
1991 7 5
1991 8 8
1991 9 6
1992 6 5
1992 7 8
1992 8 2
1992 9 2
1993 6 3
1993 7 4
1993 8 4
1993 9 3
1994 6 2
1994 7 3
1994 8 3
1994 9 1
1995 6 3
1995 7 2
1995 8 3
1995 9 6
1996 6 1
1996 7 2
1996 8 4
1996 9 7
1997 6 3
1997 7 3
1997 8 7
1997 9 5
1998 6 2
1998 7 5
1998 8 4
1998 9 3
1999 6 0
1999 7 4
1999 8 6
1999 9 9
2000 6 11
2000 7 15
2000 8 15
2000 9 8
2001 6 9
2001 7 11
2001 8 8
2001 9 15
2002 6 7
2002 7 12
2002 8 13
2002 9 14
2003 6 1
2003 7 11
2003 8 9
2003 9 7
2004 6 9
2004 7 9
2004 8 13
2004 9 14
2005 6 8
2005 7 9
2005 8 8
2005 9 13
2006 6 10
2006 7 11
2006 8 8
2006 9 6
2007 6 7
2007 7 7
2007 8 15
2007 9 6
2008 6 3
2008 7 11
2008 8 12
2008 9 11
2009 6 10
2009 7 5
2009 8 7
2009 9 10
2010 6 6
2010 7 2
2010 8 12
2010 9 6
2011 6 7
2011 7 10
2011 8 13
2011 9 9
2012 6 3
2012 7 5
2012 8 6
2012 9 3
2013 

In [39]:
#枯水期
for year in range(1986,2020):
    for month in [11,12,1,2]:
        if  month == 11:
            START_DATE  = str(year) + '-'+ str('%02d' % int(month))
            END_DATE = str(year) + '-'+ str('%02d'% (int(month)+1))
        elif month == 12:
            START_DATE  = str(year) + '-'+ str('%02d' % int(month))
            END_DATE = str(year+1) + '-'+ str('%02d'% (1))
        elif month in [1,2]:
            year1 = year+1
            START_DATE  = str(year+1) + '-'+ str('%02d' % int(month))
            END_DATE = str(year+1) + '-'+ str('%02d'% (int(month)+1))
    #get Landsat5 collction B1:blue,B2:green,B3:red,B4:NIR
        collection5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
            .filterBounds(roi) \
            .filterDate(START_DATE,END_DATE ) \
            .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'])\
            .map(addLandsatVIs)
     #get Landsat7 collction B1:blue,B2:green,B3:red,B4:NIR
        collection7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
            .filterBounds(roi) \
            .filterDate(START_DATE, END_DATE ) \
            .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'])\
            .map(addLandsatVIs)
    #get Landsat8 collction
        collection8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
            .filterBounds(roi) \
            .filterDate(START_DATE,END_DATE ) \
            .select( 
              ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa']
              ,['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'])\
            .map(addLandsatVIs)
        # Merge to a single collection 融合成一个数据集
        col1 = ee.ImageCollection(collection5.merge(collection7))
        collection = ee.ImageCollection(col1.merge(collection8))

        #names = collection.aggregate_array('system:index').getInfo()
        #print(names)
        if month in [11,12]:
            print(year,month,collection.size().getInfo())
        elif month in [1,2]:
            print(year1,month,collection.size().getInfo())

1986 11 3
1986 12 2
1987 1 4
1987 2 2
1987 11 0
1987 12 5
1988 1 2
1988 2 0
1988 11 6
1988 12 2
1989 1 4
1989 2 1
1989 11 7
1989 12 2
1990 1 2
1990 2 0
1990 11 3
1990 12 2
1991 1 1
1991 2 2
1991 11 8
1991 12 5
1992 1 4
1992 2 5
1992 11 6
1992 12 3
1993 1 3
1993 2 4
1993 11 4
1993 12 4
1994 1 6
1994 2 1
1994 11 3
1994 12 2
1995 1 3
1995 2 0
1995 11 5
1995 12 6
1996 1 2
1996 2 4
1996 11 5
1996 12 4
1997 1 4
1997 2 5
1997 11 8
1997 12 1
1998 1 3
1998 2 2
1998 11 4
1998 12 4
1999 1 0
1999 2 6
1999 11 7
1999 12 10
2000 1 7
2000 2 3
2000 11 8
2000 12 7
2001 1 11
2001 2 4
2001 11 8
2001 12 7
2002 1 5
2002 2 9
2002 11 9
2002 12 6
2003 1 10
2003 2 7
2003 11 2
2003 12 16
2004 1 12
2004 2 11
2004 11 10
2004 12 8
2005 1 8
2005 2 4
2005 11 10
2005 12 2
2006 1 7
2006 2 7
2006 11 6
2006 12 12
2007 1 7
2007 2 9
2007 11 8
2007 12 3
2008 1 3
2008 2 6
2008 11 5
2008 12 14
2009 1 9
2009 2 7
2009 11 8
2009 12 8
2010 1 6
2010 2 4
2010 11 13
2010 12 10
2011 1 11
2011 2 8
2011 11 4
2011 12 4
2012 1 1
2012 2 0